In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
--2019-04-14 13:03:37--  https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
Resolving launchpad.net (launchpad.net)... 91.189.89.222, 91.189.89.223, 2001:67c:1560:8003::8004, ...
Connecting to launchpad.net (launchpad.net)|91.189.89.222|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2019-04-14 13:03:38 ERROR 404: Not Found.

dpkg: error: cannot access archive 'google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb': No such file or directory
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
··········


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
import cv2
import numpy as np
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.datasets import load_files
from sklearn.preprocessing import LabelBinarizer
#tf.enable_eager_execution()

In [0]:
def load_data(fold_path):
  data = load_files(fold_path)
  files = np.array(data['filenames'])
  targ = np.array(data['target'])
  one_hot = LabelBinarizer()
  ohd = one_hot.fit_transform(targ)
  return files,ohd

In [0]:
def ext_im(im_file):
  img = cv2.imread(im_file)
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  img = cv2.resize(img,(256,256))
  img = img/255
  return img

In [0]:
def ext_im_gr(im_file):
  img = cv2.imread(im_file)
  img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  img = cv2.resize(img,(256,256))
  img = np.expand_dims(img,axis=2)
  img = img/255
  return img

In [0]:
def tensor_4d(fil):
  lis = [ext_im(im) for im in tqdm(fil)]
  return np.stack(lis,axis=0)

In [0]:
def tensor_4d_gr(fil):
  lis = [ext_im_gr(im) for im in tqdm(fil)]
  return np.stack(lis,axis=0)

In [0]:
def get_filter(shape,na):
  with tf.variable_scope('unet'+na,reuse=tf.AUTO_REUSE):
    w = tf.get_variable(name=na,shape=shape,dtype='float32',initializer=tf.random_normal_initializer(dtype='float32',mean=0.0,stddev=1.0))
  return w

In [0]:
def dice_cof(inp_img,pred_img):
  inp = tf.reshape(inp_img,shape=[-1])
  pred = tf.reshape(pred_img,shape=[-1])
  smooth = 1.0
  intersection = tf.reduce_sum(inp*pred)
  score = (2.* intersection + smooth)/(tf.reduce_sum(inp) + tf.reduce_sum(pred)+smooth)
  return score
  

In [0]:
def dice_loss(inp,pred):
  loss = (- dice_cof(inp,pred))
  return loss

In [0]:
def attn_blk(gating_signal,input_signal,old_filters,new_filters,name):
  
  gat_1 = tf.nn.conv2d(gating_signal,filter=get_filter(shape=[1,1,old_filters,new_filters],na=name+'_f1'),strides=[1,1,1,1],padding='SAME',name='ag_'+ name)
  gm,gv = tf.nn.moments(gat_1,axes=[0,1,2])
  g1 = tf.nn.batch_normalization(gat_1,mean=gm,variance=gv,scale=1.0,offset=0.0,variance_epsilon=0.0003,name='ag_bn_'+name)
  #print('gating',g1.shape)
  ins_1 = tf.nn.conv2d(input_signal,filter=get_filter(shape=[1,1,old_filters*2,new_filters],na=name+'_x1'),strides=[1,1,1,1],padding='SAME',name='ax_'+ name)
  im,iv = tf.nn.moments(ins_1,axes=[0,1,2])
  x1 = tf.nn.batch_normalization(ins_1,mean=im,variance=iv,scale=1.0,offset=0.0,variance_epsilon=0.0003,name='ax_bn_'+name)
  #print('att',x1.shape)
  comb_signal = tf.nn.relu(g1+x1)
  #print('comb',comb_signal.shape)
  alpha_1 = tf.nn.conv2d(comb_signal,filter=get_filter(shape=[1,1,new_filters,1],na=name+'_alpha1'),strides=[1,1,1,1],padding='SAME',name='alpha_'+ name)
  #print('alph',alpha_1.shape)
  am,av =  tf.nn.moments(alpha_1,axes=[0,1,2])
  alpha_2 =  tf.nn.batch_normalization(alpha_1,mean=am,variance=av,scale=1.0,offset=0.0,variance_epsilon=0.0003,name='alpha_bn_'+name)
  alpha = tf.nn.sigmoid(alpha_2,name='alpha_final'+name)
  #print('alph',alpha.shape)
  return alpha*input_signal

In [0]:
def attn_unet(inp):
  lay_16_1 = tf.nn.conv2d(inp,filter=get_filter(shape=[3,3,3,16],na='w_1'),strides=[1,1,1,1],padding='SAME',name='conv_16_1')
  lay_16_2 = tf.nn.relu(lay_16_1,name='re_16_1')
  m1,v1 = tf.nn.moments(lay_16_2,axes=[0,1,2])
  lay_16_3 = tf.nn.batch_normalization(lay_16_2,mean=m1,variance=v1,scale=1.0,offset=0.0,variance_epsilon=0.0003,name='bn_16')
  lay_16_4 = tf.nn.conv2d(lay_16_3,filter=get_filter(shape=[3,3,16,16],na='w_2'),strides=[1,1,1,1],padding='SAME',name='conv_16_2')
  lay_16_5 = tf.nn.relu(lay_16_4,name='re_16_2')
  #print(lay_16_5.shape)
  lay_p1 = tf.nn.max_pool(lay_16_5,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name='pool_1')
  ##128 16
  lay_32_1 = tf.nn.conv2d(lay_p1,filter=get_filter(shape=[3,3,16,32],na='w_3'),strides=[1,1,1,1],padding='SAME',name='conv_32_1')
  lay_32_2 = tf.nn.relu(lay_32_1,name='re_32_1')
  m2,v2 = tf.nn.moments(lay_32_2,axes=[0,1,2])
  lay_32_3 = tf.nn.batch_normalization(lay_32_2,mean=m2,variance=v2,scale=1.0,offset=0.0,variance_epsilon=0.0003,name='bn_32')
  lay_32_4 = tf.nn.conv2d(lay_32_3,filter=get_filter(shape=[3,3,32,32],na='w_4'),strides=[1,1,1,1],padding='SAME',name='conv_32_2')
  lay_32_5 = tf.nn.relu(lay_32_4,name='re_32_2')
  #print(lay_32_5.shape)
  lay_p2 = tf.nn.max_pool(lay_32_5,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name='pool_2')
  #64 32
  lay_64_1 = tf.nn.conv2d(lay_p2,filter=get_filter(shape=[3,3,32,64],na='w_5'),strides=[1,1,1,1],padding='SAME',name='conv_64_1')
  lay_64_2 = tf.nn.relu(lay_64_1,name='re_64_1')
  m3,v3 = tf.nn.moments(lay_64_2,axes=[0,1,2])
  lay_64_3 = tf.nn.batch_normalization(lay_64_2,mean=m3,variance=v3,scale=1.0,offset=0.0,variance_epsilon=0.0003,name='bn_64')
  lay_64_4 = tf.nn.conv2d(lay_64_3,filter=get_filter(shape=[3,3,64,64],na='w_6'),strides=[1,1,1,1],padding='SAME',name='conv_64_2')
  lay_64_5 = tf.nn.relu(lay_64_4,name='re_64_2')
  #print(lay_64_5.shape)
  lay_p3 = tf.nn.max_pool(lay_64_5,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name='pool_3')
  # 32 64
  lay_128_1 = tf.nn.conv2d(lay_p3,filter=get_filter(shape=[3,3,64,128],na='w_7'),strides=[1,1,1,1],padding='SAME',name='conv_128_1')
  lay_128_2 = tf.nn.relu(lay_128_1,name='re_128_1')
  m4,v4 = tf.nn.moments(lay_128_2,axes=[0,1,2])
  lay_128_3 = tf.nn.batch_normalization(lay_128_2,mean=m4,variance=v4,scale=1.0,offset=0.0,variance_epsilon=0.0003,name='bn_128')
  lay_128_4 = tf.nn.conv2d(lay_128_3,filter=get_filter(shape=[3,3,128,128],na='w_8'),strides=[1,1,1,1],padding='SAME',name='conv_128_2')
  lay_128_5 = tf.nn.relu(lay_128_4,name='re_128_2')
  #print(lay_128_5.shape)
  #lay_p3 = tf.nn.max_pool(lay_128_5)
  # 32 128
  up_64_1 = tf.image.resize_images(images=lay_128_5,size=[64,64],method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  #print(up_64_1.shape)
  at_64_2 = attn_blk(lay_64_4,up_64_1,64,32,'attention_1')
  #print(at_64_2.shape)
  mer_64_3 = tf.concat([lay_64_4,at_64_2],axis=-1,name='merge_1')
  #print(mer_64_3.shape)
  con_64_4 = tf.nn.conv2d(mer_64_3,filter=get_filter(shape=[3,3,192,64],na='w_9'),strides=[1,1,1,1],padding='SAME',name='up_conv1')
  con_64_5 = tf.nn.relu(con_64_4,name='rel_64_1')
  m5,v5 = tf.nn.moments(con_64_5,axes=[0,1,2])
  con_64_6 = tf.nn.batch_normalization(con_64_5,mean=m5,variance=v5,scale=1.0,offset=0.0,variance_epsilon=0.0003,name='ban_64')
  con_64_7 = tf.nn.conv2d(con_64_6,filter=get_filter(shape=[3,3,64,64],na='w_10'),strides=[1,1,1,1],padding='SAME',name='up_conv2')
  con_64_8 = tf.nn.relu(con_64_7,name='rel_64_2')
  
  up_32_1 = tf.image.resize_images(images=con_64_8,size=[128,128],method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  at_32_2 = attn_blk(lay_32_4,up_32_1,32,16,'attention_2')
  mer_32_3 = tf.concat([lay_32_5,at_32_2],axis=-1,name='merge_2')
  con_32_4 = tf.nn.conv2d(mer_32_3,filter=get_filter(shape=[3,3,96,32],na='w_11'),strides=[1,1,1,1],padding='SAME',name='up_conv3')
  con_32_5 = tf.nn.relu(con_32_4,name='rel_32_1')
  m6,v6 = tf.nn.moments(con_32_5,axes=[0,1,2])
  con_32_6 = tf.nn.batch_normalization(con_32_5,mean=m6,variance=v6,scale=1.0,offset=0.0,variance_epsilon=0.0003,name='ban_32')
  con_32_7 = tf.nn.conv2d(con_32_6,filter=get_filter(shape=[3,3,32,32],na='w_12'),strides=[1,1,1,1],padding='SAME',name='up_conv4')
  con_32_8 = tf.nn.relu(con_32_7,name='rel_32_2')
  
  up_16_1 = tf.image.resize_images(images=con_32_8,size=[256,256],method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  at_16_2 = attn_blk(lay_16_4,up_16_1,16,8,'attention_3')
  mer_16_3 = tf.concat([lay_16_5,at_16_2],axis=-1,name='merge_3')
  con_16_4 = tf.nn.conv2d(mer_16_3,filter=get_filter(shape=[3,3,48,16],na='w_13'),strides=[1,1,1,1],padding='SAME',name='up_conv5')
  con_16_5 = tf.nn.relu(con_16_4,name='rel_16_1')
  m7,v7 = tf.nn.moments(con_16_5,axes=[0,1,2])
  con_16_6 = tf.nn.batch_normalization(con_16_5,mean=m7,variance=v7,scale=1.0,offset=0.0,variance_epsilon=0.0003,name='ban_16')
  con_16_7 = tf.nn.conv2d(con_16_6,filter=get_filter(shape=[3,3,16,16],na='w_14'),strides=[1,1,1,1],padding='SAME',name='up_conv6')
  con_16_8 = tf.nn.relu(con_16_7,name='rel_16_2')
  
  fin_img = tf.nn.conv2d(con_16_8,filter=get_filter(shape=[3,3,16,1],na='w_15'),strides=[1,1,1,1],padding='SAME',name='final_img')
  return fin_img

In [0]:
img_fold = 'drive/datasets/ISBI/images'
lab_fold = 'drive/datasets/ISBI/labels'
tes_img_fold = 'drive/datasets/ISBI/test/images'
tes_lab_fold = 'drive/datasets/ISBI/test/labels'

In [0]:
img_files,_ = load_data(img_fold)

In [0]:
lab_files,_ = load_data(lab_fold)

In [0]:
tes_img_files,_ = load_data(tes_img_fold)

In [0]:
tes_lab_files,_ = load_data(tes_lab_fold)

In [0]:
imgs = tensor_4d(img_files)
tes_imgs = tensor_4d(tes_img_files)
labs = tensor_4d_gr(lab_files)
tes_labs = tensor_4d_gr(tes_lab_files)

100%|██████████| 30/30 [00:00<00:00, 166.69it/s]


In [0]:
ip = tf.placeholder(shape=[None,256,256,3],dtype='float32',name='input_img')
op = tf.placeholder(shape=[None,256,256,1],dtype='float32',name='output_img')

In [0]:
mod = attn_unet(ip)

In [0]:
fin_out = tf.nn.sigmoid(mod,name='sigmoid')
#los,los_op = tf.metrics.mean_iou(labels=op,predictions=mod,num_classes=1)
los = tf.nn.sigmoid_cross_entropy_with_logits(logits=mod,labels=op)
los_bce = tf.reduce_mean(los)
los_dice = tf.reduce_mean(dice_loss(op,fin_out))
los_m = tf.log(los_dice)
#los_m = tf.reduce_mean(op * (-tf.log(mod)) + (1 - op) * (-tf.log(1 - mod)),axis=1)
#los = tf.reduce_mean(tf.square(mod-op))
opt = tf.train.AdamOptimizer(learning_rate=0.001)
train = opt.minimize(los_m)
init = tf.global_variables_initializer()

Instructions for updating:
Use tf.cast instead.


In [0]:
with tf.Session() as sess:
  sess.run(init)
  for epoch in tqdm(range(80)):
    batch = 0
    for start,end in zip(range(0,len(imgs),3),range(3,len(imgs),3)):
      xtr = imgs[start:end]
      ytr = labs[start:end]
      #print(xtr.shape)
      #x = sess.run(mod,feed_dict={ip:xtr})
      #print((los.shape))
      sess.run(train,feed_dict={ip:xtr,op:ytr})
      lo = sess.run(los_m,feed_dict={ip:xtr,op:ytr})
      #print('epoch:',epoch,'batch:',batch,'loss:',lo)
      batch = batch + 1
    val_loss = sess.run(los_m,feed_dict={ip:tes_imgs,op:tes_labs})
    #print('epoch:',epoch,'val loss:',val_loss)

100%|██████████| 80/80 [02:09<00:00,  1.61s/it]
